## 11.2 파이썬은 시퀀스를 찾아낸다.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f:
    print(i)

0
10
20


In [4]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

In [5]:
from random import shuffle
l = list(range(10))
shuffle(l)

In [8]:
l

[6, 8, 5, 2, 4, 9, 1, 0, 3, 7]

In [9]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [11]:
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='4', suit='hearts'),
 Card(rank='J', suit='diamonds'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='J', suit='spades')]

## 11.4 알렉스 마르텔리의 물새